In [36]:
 !
zip -r example.zip original_folder

SyntaxError: invalid syntax (3403750767.py, line 2)

In [7]:
import os
import getpass

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvapi_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key
    os.environ["NGC_API_KEY"] = nvapi_key


Enter your NVIDIA API key:  ········


In [3]:
import os
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from pydantic.v1 import BaseModel
from langchain_nvidia_ai_endpoints._statics import Model, register_model

In [9]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

In [10]:
ChatNVIDIA.get_available_models()

[Model(id='adept/fuyu-8b', model_type='vlm', client='ChatNVIDIA', endpoint='https://ai.api.nvidia.com/v1/vlm/adept/fuyu-8b', aliases=['ai-fuyu-8b', 'playground_fuyu_8b', 'fuyu_8b'], supports_tools=False, base_model=None),
 Model(id='google/gemma-7b', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-gemma-7b', 'playground_gemma_7b', 'gemma_7b'], supports_tools=False, base_model=None),
 Model(id='nvidia/llama3-chatqa-1.5-8b', model_type='qa', client='ChatNVIDIA', endpoint=None, aliases=['ai-chatqa-1.5-8b'], supports_tools=False, base_model=None),
 Model(id='01-ai/yi-large', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-yi-large'], supports_tools=False, base_model=None),
 Model(id='nvidia/llama3-chatqa-1.5-70b', model_type='qa', client='ChatNVIDIA', endpoint=None, aliases=['ai-chatqa-1.5-70b'], supports_tools=False, base_model=None),
 Model(id='databricks/dbrx-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-dbrx-instr

<h3>
 Hands-on section    
</h3>

<h3>Initialising "meta / llama-3.1-8b-instruct" LLM model to the cluster</h3>

In [11]:
! docker ps 

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [12]:
! echo -e "$NGC_API_KEY" | docker login nvcr.io --username '$oauthtoken' --password-stdin

WARNING! Your password will be stored unencrypted in /home/gsh-3atzc7/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [13]:
! docker pull nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.2

1.1.2: Pulling from nim/meta/llama-3.1-8b-instruct
Digest: sha256:c53ac18b1161899e3b7006e4cb24737c218f90c320ca2552a5d75e75d9fc4931
Status: Image is up to date for nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.2
nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.2


In [22]:
! docker image ls -a --no-trunc


REPOSITORY                                     TAG       IMAGE ID                                                                  CREATED        SIZE
nvcr.io/nim/meta/llama-3.1-8b-instruct         1.1.2     sha256:c9102c3e95f3acae5b41fdafd7aefd3389bdaf33cf0b3458041e45046bf6bb1d   2 months ago   12.9GB
nvcr.io/nim/nvidia/nv-rerankqa-mistral-4b-v3   1.0.2     sha256:47df3d3998eb2f8fcf3da6c8aa55297b34c4789234ea0f2cbf54b4bcd411b37d   2 months ago   15.5GB
nvcr.io/nim/nvidia/nv-embedqa-e5-v5            1.0.1     sha256:fa5c1fc5ccb39cd6d9910ac04fb8bda7ea46e6f82de8d05c2972d1071d5a62c7   2 months ago   15.7GB


In [4]:
from os.path import expanduser
home = expanduser("~")
os.environ['LOCAL_NIM_CACHE']=f"{home}/.cache/nim"
!echo $LOCAL_NIM_CACHE

/home/gsh-3atzc7/.cache/nim


In [5]:
!mkdir -p "$LOCAL_NIM_CACHE"
!chmod 777 "$LOCAL_NIM_CACHE"

In [ ]:
import random
import socket

def find_available_port(start=11000, end=11999):
    while True:
        # Randomly select a port between start and end range
        port = random.randint(start, end)
        
        # Try to create a socket and bind to the port
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            try:
                sock.bind(("localhost", port))
                # If binding is successful, the port is free
                return port
            except OSError:
                # If binding fails, the port is in use, continue to the next iteration
                continue

# Find and print an available port
os.environ['CONTAINER_PORT'] = str(find_available_port())
print(f"Your have been alloted the available port: {os.environ['CONTAINER_PORT']}")

In [14]:
! docker run -it -d --rm \
--gpus '"device=1,2"' \
--name=llm_nim \
--shm-size=32GB \
-e NGC_API_KEY \
-e NIM_MODEL_PROFILE="7ea3369b85d7aee24e0739df829da8832b6873803d5f5aca490edad7360830c8" \
-v $LOCAL_NIM_CACHE:/opt/nim/.cache \
-u $(id -u) \
-p 8000:8000 \
nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.2

# In order to ensure, the local NIM container is completely loaded and doesn't remain in pending stage, we instantiate a wait interval
! sleep 60

9f0158ebdf3f4a37959de1a5da17285c25f3a02eb462dad502b9b146a955627a


In [24]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(base_url= "http://0.0.0.0:8000/v1", model="meta/llama-3.1-8b-instruct", temperature=0.1, max_tokens=1000, top_p=1.0)

result = llm.invoke("What is the capital of France?")
print(result.content)

The capital of France is Paris.


In [56]:
!curl -X 'POST' \
    "http://0.0.0.0:8000/v1/completions" \
    -H "accept: application/json" \
    -H "Content-Type: application/json" \
    -d '{"model": "meta/llama-3.1-8b-instruct", "prompt": "What is the capital of France?", "max_tokens": 64}'

{"id":"cmpl-841c3b6fea2d4ec8a3db6ae681e1c24a","object":"text_completion","created":1728051640,"model":"meta/llama-3.1-8b-instruct","choices":[{"index":0,"text":" [Answer]\nThe capital of France is Paris.\nLooking for somewhere to eat in Paris?\nWhy the Eiffel Tower is a must visit.\nPractical information about visiting France. [Answer]\nLanguage : French\nCurrency: Euro\nVisa requirements: Check with your home country's government on the current visa requirements\n","logprobs":null,"finish_reason":"length","stop_reason":null}],"usage":{"prompt_tokens":8,"total_tokens":72,"completion_tokens":64}}

In [15]:
! docker ps


CONTAINER ID   IMAGE                                                COMMAND                  CREATED              STATUS              PORTS                                         NAMES
9f0158ebdf3f   nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.2         "/opt/nvidia/nvidia_…"   About a minute ago   Up About a minute   0.0.0.0:8000->8000/tcp, :::8000->8000/tcp     llm_nim
9f6e021a042b   nvcr.io/nim/nvidia/nv-rerankqa-mistral-4b-v3:1.0.2   "/opt/nvidia/nvidia_…"   24 hours ago         Up 24 hours         0.0.0.0:11737->8000/tcp, :::11737->8000/tcp   rerank_nim
6c869ca42d4d   nvcr.io/nim/nvidia/nv-embedqa-e5-v5:1.0.1            "/opt/nvidia/nvidia_…"   24 hours ago         Up 24 hours         0.0.0.0:11022->8000/tcp, :::11022->8000/tcp   embed_nim


In [20]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)




NameError: name 'model' is not defined

In [13]:
!docker container stop llm_nim

llm_nim


In [ ]:
!docker container stop embed_nim

In [35]:
!docker run nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.2 list-model-profiles


== NVIDIA Inference Microservice LLM NIM ==

NVIDIA Inference Microservice LLM NIM Version 1.1.2
Model: nim/meta/llama-3.1-8b-instruct

Container image Copyright (c) 2016-2024, NVIDIA CORPORATION & AFFILIATES. All rights reserved.

The use of this model is governed by the NVIDIA AI Foundation Models Community License Agreement (found at https://www.nvidia.com/en-us/agreements/enterprise-software/nvidia-ai-foundation-models-community-license-agreement/#:~:text=This%20license%20agreement%20(%E2%80%9CAgreement%E2%80%9D,algorithms%2C%20parameters%2C%20configuration%20files%2C).

ADDITIONAL INFORMATION: Llama 3.1 Community License Agreement, Built with Llama.

   Use the NVIDIA Container Toolkit to start this container with GPU support; see
   https://docs.nvidia.com/datacenter/cloud-native/ .

INFO 10-16 13:10:43.826 info.py:23] Unable to get hardware specifications, retrieving every profile's info.
0494aafce0df9eeaea49bbca6b25fc3013d0e8a752ebcf191a2ddeaab19481ee: tensorrt_llm-l40s-bf16-t

In [ ]:
! docker ps


In [ ]:
! docker ls

In [ ]:
from os.path import expanduser
home = expanduser("~")
os.environ['LOCAL_NIM_CACHE']=f"{home}/.cache/nim"
!echo $LOCAL_NIM_CACHE

In [ ]:
!mkdir -p "$LOCAL_NIM_CACHE"
# !chmod 777 "$LOCAL_NIM_CACHE"

In [ ]:
import random
import socket

def find_available_port(start=11000, end=11999):
    while True:
        # Randomly select a port between start and end range
        port = random.randint(start, end)
        
        # Try to create a socket and bind to the port
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            try:
                sock.bind(("localhost", port))
                # If binding is successful, the port is free
                return port
            except OSError:
                # If binding fails, the port is in use, continue to the next iteration
                continue

# Find and print an available port
os.environ['CONTAINER_PORT'] = str(find_available_port())
print(f"Your have been alloted the available port: {os.environ['CONTAINER_PORT']}")

In [7]:
! docker run -it -d --rm \
--gpus device=2 \
--name=embed_nim \
--shm-size=16GB  \
-e NGC_API_KEY \
-v $LOCAL_NIM_CACHE:/opt/nim/.cache \
-u $(id -u) \
-p 11022:8000 \
nvcr.io/nim/nvidia/nv-embedqa-e5-v5:1.0.1

# In order to ensure, the local NIM container is completely loaded and doesn't remain in pending stage, we instantiate a wait interval
! sleep 60

6c869ca42d4d914c99ced69bd17e7e8d69c5d16ae5436af2b0f910e1c5f82e68


In [27]:
!docker ps


CONTAINER ID   IMAGE                                          COMMAND                  CREATED              STATUS              PORTS                                         NAMES
5fee8a1afdc8   nvcr.io/nim/nvidia/nv-embedqa-e5-v5:1.0.1      "/opt/nvidia/nvidia_…"   About a minute ago   Up About a minute   0.0.0.0:11022->8000/tcp, :::11022->8000/tcp   embed_nim
78bbd2d69c09   nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.2   "/opt/nvidia/nvidia_…"   8 minutes ago        Up 8 minutes        0.0.0.0:8000->8000/tcp, :::8000->8000/tcp     llm_nim


In [ ]:
!docker container stop embed_nim

<h3>Deploying embedding and retriever models locally</h3>

In [ ]:
! docker pull nvcr.io/nim/nvidia/nv-rerankqa-mistral-4b-v3:1.0.2

In [2]:
from os.path import expanduser
home = expanduser("~")
os.environ['LOCAL_NIM_CACHE']=f"{home}/.cache/nim"
!echo $LOCAL_NIM_CACHE

NameError: name 'os' is not defined

In [ ]:
!mkdir -p "$LOCAL_NIM_CACHE"
# !chmod 777 "$LOCAL_NIM_CACHE"

In [ ]:
import random
import socket

def find_available_port(start=11000, end=11999):
    while True:
        # Randomly select a port between start and end range
        port = random.randint(start, end)
        
        # Try to create a socket and bind to the port
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            try:
                sock.bind(("localhost", port))
                # If binding is successful, the port is free
                return port
            except OSError:
                # If binding fails, the port is in use, continue to the next iteration
                continue

# Find and print an available port
os.environ['CONTAINER_PORT'] = str(find_available_port())
print(f"Your have been alloted the available port: {os.environ['CONTAINER_PORT']}")

In [8]:
! docker run -it -d --rm \
--gpus device=0  \
--name=rerank_nim \
--shm-size=16GB  \
-e NGC_API_KEY \
-v $LOCAL_NIM_CACHE:/opt/nim/.cache \
-u $(id -u) \
-p 11737:8000 \
nvcr.io/nim/nvidia/nv-rerankqa-mistral-4b-v3:1.0.2

9f6e021a042b54291fcc9b160a33dc5e99f3aef1e8ca22bc5e8c8310d38fe6aa


In [9]:
!docker ps

CONTAINER ID   IMAGE                                                COMMAND                  CREATED          STATUS          PORTS                                         NAMES
9f6e021a042b   nvcr.io/nim/nvidia/nv-rerankqa-mistral-4b-v3:1.0.2   "/opt/nvidia/nvidia_…"   56 seconds ago   Up 54 seconds   0.0.0.0:11737->8000/tcp, :::11737->8000/tcp   rerank_nim
6c869ca42d4d   nvcr.io/nim/nvidia/nv-embedqa-e5-v5:1.0.1            "/opt/nvidia/nvidia_…"   2 minutes ago    Up 2 minutes    0.0.0.0:11022->8000/tcp, :::11022->8000/tcp   embed_nim
dad2c034c5d2   nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.2         "/opt/nvidia/nvidia_…"   11 minutes ago   Up 11 minutes   0.0.0.0:8000->8000/tcp, :::8000->8000/tcp     llm_nim


In [57]:
urls = ["https://www.nvidia.com/en-in/glossary/retrieval-augmented-generation/",
       "https://en.wikipedia.org/wiki/Retrieval-augmented_generation",
        "https://docs.nvidia.com/cuda/",
        "https://github.com/NVIDIA/cuda-samples"
        "https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html"
       ]

In [58]:
import re
import requests
from bs4 import BeautifulSoup
from typing import List, Union

def html_document_loader(url: Union[str, bytes]) -> str:
    """
    Loads the HTML content of a document from a given URL and return it's content.

    Args:
        url: The URL of the document.

    Returns:
        The content of the document.

    Raises:
        Exception: If there is an error while making the HTTP request.

    """
    try:
        response = requests.get(url)
        html_content = response.text
    except Exception as e:
        print(f"Failed to load {url} due to exception {e}")
        return ""

    try:
        # Create a Beautiful Soup object to parse html
        soup = BeautifulSoup(html_content, "html.parser")

        # Remove script and style tags
        for script in soup(["script", "style"]):
            script.extract()

        # Get the plain text from the HTML document
        text = soup.get_text()

        # Remove excess whitespace and newlines
        text = re.sub("\s+", " ", text).strip()

        return text
    except Exception as e:
        print(f"Exception {e} while loading document")
        return ""

In [59]:
def create_embeddings(embeddings_model,embedding_path: str = "./embed"):

    embedding_path = "./embed"
    print(f"Storing embeddings to {embedding_path}")

    documents = []
    for url in urls:
        document = html_document_loader(url)
        documents.append(document)


    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=0,
        length_function=len,
    )
    print("Total documents:",len(documents))
    texts = text_splitter.create_documents(documents)
    print("Total texts:",len(texts))
    index_docs(embeddings_model,url, text_splitter, texts, embedding_path,)
    print("Generated embedding successfully")

In [26]:
embeddings_model = NVIDIAEmbeddings(base_url="http://0.0.0.0:11022/v1", model='nvidia/nv-embedqa-e5-v5')

<h3>Faiss vector store through langchain</h3>

In [15]:
from typing import List, Union
import os
from langchain.vectorstores import FAISS
# model_kwargs = {"device": "cuda"}

def index_docs(embeddings_model, url: Union[str, bytes], splitter, documents: List[str], dest_embed_dir: str) -> None:
    """
    Split the documents into chunks and create embeddings for them.
    
    Args:
        embeddings_model: Model used for creating embeddings.
        url: Source url for the documents.
        splitter: Splitter used to split the documents.
        documents: List of documents whose embeddings need to be created.
        dest_embed_dir: Destination directory for embeddings.
    """
    texts = []
    metadatas = []

    for document in documents:
        chunk_texts = splitter.split_text(document.page_content)
        texts.extend(chunk_texts)
        metadatas.extend([document.metadata] * len(chunk_texts))

    if os.path.exists(dest_embed_dir):
        docsearch = FAISS.load_local(
            folder_path=dest_embed_dir, 
            embeddings=embeddings_model, 
            allow_dangerous_deserialization=True
        )
        docsearch.add_texts(texts, metadatas=metadatas)
    else:
        docsearch = FAISS.from_texts(texts, embedding=embeddings_model, metadatas=metadatas,)

    docsearch.save_local(folder_path=dest_embed_dir)

In [16]:
%%time
create_embeddings(embeddings_model=embeddings_model)

Storing embeddings to ./embed
Processing PDF: CUDA_C_Programming_Guide.pdf
Total documents: 6
Total texts: 3026
Generated embedding successfully
CPU times: user 2.67 s, sys: 181 ms, total: 2.85 s
Wall time: 10.4 s


In [21]:
import faiss
print(faiss.get_num_gpus())

3


In [17]:
# load Embed documents
embedding_path = "./embed/"
docsearch = FAISS.load_local(folder_path=embedding_path, embeddings=embeddings_model, allow_dangerous_deserialization=True)

<h3>HTML and pdf embedding </h3>

In [33]:
import os
import re
import requests
import pymupdf  # Explicitly using pymupdf instead of fitz
from bs4 import BeautifulSoup
from typing import List, Union
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
  # Replace with your actual embeddings model
import faiss
import numpy as np
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# List of URLs
urls = [
    "https://www.nvidia.com/en-in/glossary/retrieval-augmented-generation/",
    "https://en.wikipedia.org/wiki/Retrieval-augmented_generation",
    "https://docs.nvidia.com/cuda/",
    "https://github.com/NVIDIA/cuda-samples",
    "https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html",
    "https://developer.nvidia.com/blog/efficient-cuda-debugging-using-compute-sanitizer-with-nvtx-and-creating-custom-tools/",
    "https://developer.nvidia.com/blog/debugging-cuda-more-efficiently-with-nvidia-compute-sanitizer/",
    "https://developer.nvidia.com/blog/multi-gpu-programming-with-standard-parallel-c-part-1/",
    "https://developer.nvidia.com/blog/multi-gpu-programming-with-standard-parallel-c-part-2",
    "https://developer.nvidia.com/blog/efficient-cuda-debugging-memory-initialization-and-thread-synchronization-with-nvidia-compute-sanitizer/",
    "https://cuda-tutorial.readthedocs.io/en/latest/tutorials/tutorial02/#introduction",
    "https://developer.nvidia.com/blog/even-easier-introduction-cuda/",
    "https://github.com/jkonvicka/Nvidia-CUDA-course/blob/main/AC_CUDA_C.md",
    "https://github.com/jkonvicka/Nvidia-CUDA-course/blob/main/cuda_cheatsheet.md",
    "https://github.com/jkonvicka/Nvidia-CUDA-course/blob/main/Unified%20Memory.md",
    "https://github.com/jkonvicka/Nvidia-CUDA-course/blob/main/Streaming%20and%20Visual%20Profiling.md",
    "https://github.com/jkonvicka/Nvidia-CUDA-course/blob/main/AC_CUDA_C.md",
    "https://tbetcke.github.io/hpc_lecture_notes/intro.html",
]

# Directory containing PDF files
pdf_dir = '/mnt/lustre/hackathons/hack_teams/hack_team_16/workspace/Jishnu/Pdf'

# Directory containing other file types (e.g., .txt, .docx)
other_files_dir = '/mnt/lustre/hackathons/hack_teams/hack_team_16/workspace/Jishnu/text'

# Function to load HTML content from a URL
def html_document_loader(url: Union[str, bytes]) -> str:
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for HTTP errors
        html_content = response.text
    except Exception as e:
        logging.error(f"Failed to load {url} due to exception: {e}")
        return ""

    try:
        soup = BeautifulSoup(html_content, "html.parser")
        # Remove script and style tags
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text()
        text = re.sub(r"\s+", " ", text).strip()
        return text if isinstance(text, str) else ""
    except Exception as e:
        logging.error(f"Exception {e} while parsing HTML from {url}")
        return ""

# Function to extract text from PDFs using PyMuPDF
def extract_text_from_pdfs(pdf_directory: str) -> List[dict]:
    pdf_texts = []
    
    for pdf_file in os.listdir(pdf_directory):
        if pdf_file.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_directory, pdf_file)
            logging.info(f"Processing PDF: {pdf_file}")

            # Open the PDF file
            try:
                with pymupdf.open(pdf_path) as doc:
                    extracted_text = ""
                    for page_num in range(len(doc)):
                        page = doc.load_page(page_num)
                        extracted_text += page.get_text()
            except Exception as e:
                logging.error(f"Failed to process PDF {pdf_file}: {e}")
                continue

            # Ensure extracted text is a string
            extracted_text = extracted_text if isinstance(extracted_text, str) else ""
            
            # Append extracted text and metadata
            pdf_texts.append({
                'page_content': extracted_text,
                'metadata': {'type': 'pdf', 'source': pdf_file}
            })

    return pdf_texts

# Function to handle other file types using UnstructuredFileLoader
def extract_text_from_other_files(file_directory: str) -> List[dict]:
    other_texts = []
    
    for file_name in os.listdir(file_directory):
        file_path = os.path.join(file_directory, file_name)
        # Use UnstructuredFileLoader for other file types
        loader = UnstructuredFileLoader(file_path)
        try:
            loaded_documents = loader.load()
            for doc in loaded_documents:
                other_texts.append({
                    'page_content': doc.page_content,
                    'metadata': {'type': 'file', 'source': file_name}
                })
        except Exception as e:
            logging.error(f"Failed to load file {file_name} due to exception: {e}")
    
    return other_texts

# Function to embed documents in batches
def embed_documents_batch(embeddings_model, documents: List[str], batch_size: int = 32) -> np.ndarray:
    """
    Embed documents in batches and return a NumPy array of embeddings.
    
    Args:
        embeddings_model: The embeddings model instance.
        documents (List[str]): A list of document texts to embed.
        batch_size (int): Number of documents to process in each batch.
    
    Returns:
        np.ndarray: A 2D NumPy array where each row is an embedding vector.
    """
    embeddings = []
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        batch_embeddings = embeddings_model.embed_documents(batch)
        
        # Convert to NumPy array if it's a list
        if isinstance(batch_embeddings, list):
            batch_embeddings = np.array(batch_embeddings, dtype='float32')
        elif isinstance(batch_embeddings, np.ndarray):
            batch_embeddings = batch_embeddings.astype('float32')
        else:
            raise TypeError("embed_documents must return a list or a NumPy array.")
        
        embeddings.append(batch_embeddings)
    
    # Concatenate all batches into a single NumPy array
    return np.vstack(embeddings)

# Function to index documents using embeddings with FAISS GPU
def index_docs(
    embeddings_model, 
    urls: List[str], 
    splitter, 
    documents: List[object], 
    dest_embed_dir: str
) -> None:
    """
    Index documents using FAISS with GPU support.
    
    Args:
        embeddings_model: The embeddings model instance.
        urls (List[str]): List of URLs processed.
        splitter: Text splitter instance.
        documents (List[object]): List of Document objects.
        dest_embed_dir (str): Directory to save/load the FAISS index.
    """
    texts = []
    metadatas = []

    for document in documents:
        # Access attributes instead of dictionary keys
        page_content = document.page_content
        metadata = document.metadata

        # Ensure document content is a string
        if not isinstance(page_content, str):
            logging.warning(f"Skipping document due to invalid content: {document}")
            continue

        # Split the text into chunks
        chunk_texts = splitter.split_text(page_content)
        texts.extend(chunk_texts)
        metadatas.extend([metadata] * len(chunk_texts))

    if not texts:
        logging.error("No valid texts found for embedding.")
        return

    # Embed the texts in batches
    try:
        embeddings = embed_documents_batch(embeddings_model, texts, batch_size=32)
        logging.info(f"Generated embeddings for {len(texts)} text chunks.")
    except Exception as e:
        logging.error(f"Failed to generate embeddings: {e}")
        return

    # Determine the embedding dimension from the first embedding
    dim = embeddings.shape[1]
    logging.info(f"Embedding dimension: {dim}")

    # Initialize FAISS with GPU support
    try:
        index = faiss.IndexFlatL2(dim)  # Exact search index
        res = faiss.StandardGpuResources()
        gpu_index = faiss.index_cpu_to_gpu(res, 0, index)  # GPU ID 0
        logging.info("FAISS index moved to GPU 0.")
    except Exception as e:
        logging.error(f"Failed to initialize FAISS index on GPU: {e}")
        return

    # Create FAISS vector store using Langchain
    try:
        if os.path.exists(dest_embed_dir):
            # Load existing FAISS index
            docsearch = FAISS.load_local(
                folder_path=dest_embed_dir, 
                embeddings=embeddings_model, 
                allow_dangerous_deserialization=True
            )
            logging.info("Loaded existing FAISS index from local directory.")
            
            # Extract vectors from the existing index
            cpu_index = docsearch.index
            vectors = cpu_index.reconstruct_n(0, cpu_index.ntotal)
            gpu_index.add(vectors)
            logging.info(f"Added {cpu_index.ntotal} vectors to GPU FAISS index.")
        else:
            # Add embeddings to the GPU index
            gpu_index.add(embeddings)
            logging.info(f"Added {len(embeddings)} vectors to FAISS GPU index.")

            # Create FAISS vector store using Langchain
            docsearch = FAISS.from_texts(
                texts, 
                embedding=embeddings_model, 
                metadatas=metadatas, 
                index=gpu_index
            )
            logging.info("Created new FAISS vector store with GPU index.")
        
        # Save the FAISS index locally
        docsearch.save_local(folder_path=dest_embed_dir)
        logging.info(f"FAISS index saved to {dest_embed_dir}.")
    except Exception as e:
        logging.error(f"Failed during FAISS vector store creation or saving: {e}")
        return

# Function to process HTML, PDF, and other files and create embeddings
def create_embeddings(embeddings_model, embedding_path: str = "./embed"):
    """
    Process documents and create embeddings using FAISS with GPU support.
    
    Args:
        embeddings_model: The embeddings model instance.
        embedding_path (str): Directory to save/load the FAISS index.
    """
    logging.info(f"Storing embeddings to {embedding_path}")

    documents = []
    total_html_docs = 0  # Variable to keep track of HTML document count

    # Load HTML content from URLs
    for url in urls:
        document_content = html_document_loader(url)
        if document_content:
            total_html_docs += 1
            documents.append({
                'page_content': document_content, 
                'metadata': {'type': 'html', 'source': url}
            })

    # Extract text from PDFs
    pdf_documents = extract_text_from_pdfs(pdf_dir)
    total_pdf_docs = len(pdf_documents)  # Count PDF documents
    documents.extend(pdf_documents)

    # Check if the directory for other files exists and is not empty
    if os.path.exists(other_files_dir) and os.listdir(other_files_dir):
        other_files_documents = extract_text_from_other_files(other_files_dir)
        documents.extend(other_files_documents)

    # Extract the text content for splitting
    document_texts = [doc['page_content'] for doc in documents if isinstance(doc.get('page_content', ""), str)]

    # Split text into chunks and create embeddings
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=0,
        length_function=len,
    )
    logging.info(f"Total HTML documents: {total_html_docs}")
    logging.info(f"Total PDF documents: {total_pdf_docs}")
    logging.info(f"Total other documents: {len(documents) - total_html_docs - total_pdf_docs}")
    logging.info(f"Total documents: {len(document_texts)}")

    # Create Langchain Document objects
    texts = text_splitter.create_documents(document_texts)  # This returns a list of Document objects
    logging.info(f"Total texts (chunks): {len(texts)}")

    # Index the documents
    index_docs(embeddings_model, urls, text_splitter, texts, embedding_path)
    logging.info("Generated embedding successfully.")

# Example usage
if __name__ == "__main__":
    # Initialize your embeddings model
    embeddings_model = embeddings_model # Replace with NVIDIAEmbeddings or your specific model

    # If using NVIDIAEmbeddings as per your input, ensure it's correctly implemented
    # For example:
    # embeddings_model = NVIDIAEmbeddings(base_url="http://0.0.0.0:11022/v1", model='nvidia/nv-embedqa-e5-v5')

    # Optionally, wrap embed_query and embed_documents if they return lists
    def embed_query_np(query):
        embedding = embeddings_model.embed_query(query)
        if isinstance(embedding, list):
            return np.array(embedding, dtype='float32')
        elif isinstance(embedding, np.ndarray):
            return embedding.astype('float32')
        else:
            raise TypeError("embed_query must return a list or a NumPy array.")

    def embed_documents_np(documents):
        embeddings = embeddings_model.embed_documents(documents)
        if isinstance(embeddings, list):
            return np.array(embeddings, dtype='float32')
        elif isinstance(embeddings, np.ndarray):
            return embeddings.astype('float32')
        else:
            raise TypeError("embed_documents must return a list or a NumPy array.")

    # Replace the methods in embeddings_model with the wrapped ones
    embeddings_model.embed_query = embed_query_np
    embeddings_model.embed_documents = embed_documents_np

    # Create embeddings
    create_embeddings(embeddings_model, embedding_path=pdf_dir)  # Specify your embedding path


ValueError: "NVIDIAEmbeddings" object has no field "embed_query"

In [34]:
%%time
embeddings_model = NVIDIAEmbeddings(base_url="http://0.0.0.0:11022/v1", model='nvidia/nv-embedqa-e5-v5')
create_embeddings(embeddings_model=embeddings_model)

INFO:root:Storing embeddings to ./embed
INFO:root:Processing PDF: 9781788996242_ColorImages.pdf
INFO:root:Processing PDF: CUDA_C_Programming_Guide.pdf
INFO:root:Processing PDF: [2013] The_CUDA_Handbook.pdf
INFO:root:Processing PDF: sc11-cuda-c-basics.pdf
INFO:root:Processing PDF: openacc-guide.pdf
INFO:root:Processing PDF: CUDA_by_Example.pdf
INFO:root:Total HTML documents: 18
INFO:root:Total PDF documents: 6
INFO:root:Total other documents: 0
INFO:root:Total documents: 24
INFO:root:Total texts (chunks): 6812
INFO:root:Generated embeddings for 6812 text chunks.
INFO:root:Embedding dimension: 1024
INFO:root:FAISS index moved to GPU 0.
INFO:root:Loaded existing FAISS index from local directory.
INFO:root:Added 49073 vectors to GPU FAISS index.
INFO:root:FAISS index saved to ./embed.
INFO:root:Generated embedding successfully.


CPU times: user 7.14 s, sys: 727 ms, total: 7.87 s
Wall time: 23.7 s


In [46]:
!docker ps -s


CONTAINER ID   IMAGE                                                COMMAND                  CREATED        STATUS        PORTS                                         NAMES        SIZE
4ad7b8ee2bb5   nvcr.io/nim/nvidia/nv-embedqa-e5-v5:1.0.1            "/opt/nvidia/nvidia_…"   21 hours ago   Up 21 hours   0.0.0.0:11022->8000/tcp, :::11022->8000/tcp   embed_nim    995kB (virtual 15.7GB)
6ef6ad73da55   nvcr.io/nim/nvidia/nv-rerankqa-mistral-4b-v3:1.0.2   "/opt/nvidia/nvidia_…"   21 hours ago   Up 21 hours   0.0.0.0:11737->8000/tcp, :::11737->8000/tcp   rerank_nim   1.91MB (virtual 15.5GB)
a5732a8ccde9   nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.2         "/opt/nvidia/nvidia_…"   22 hours ago   Up 22 hours   0.0.0.0:8000->8000/tcp, :::8000->8000/tcp     LLM_nim      9.03MB (virtual 12.9GB)


In [40]:
!docker rmi 3cb29b0d79e6

Untagged: nvcr.io/nim/meta/llama3-8b-instruct:1.0.0
Untagged: nvcr.io/nim/meta/llama3-8b-instruct@sha256:7fe6071923b547edd9fba87c891a362ea0b4a88794b8a422d63127e54caa6ef7
Deleted: sha256:3cb29b0d79e6d84ca4fd132aa66b652408e8c0fb88177cfee06a55418fdc3de2
Deleted: sha256:d363b236bc57a7c2d43748a0272361d9c93a1a830b1b65729d48070734919bf4
Deleted: sha256:0afef4f6b6a5eae810e2dde7db5d571d9ac4ac120f1ab8938b6d25ba923d85b4
Deleted: sha256:113fe49b1d137ede04be15d2eca24351c065822d01a5a6c65410a3b37660784e
Deleted: sha256:90cc92b781e2986bdfff11a03227a7133a237d376e6ae3ada86d28cf710effac
Deleted: sha256:c94eb581f0fdd36f8419033ec5c3791925899c00c75638d88ce8638e959df5c6
Deleted: sha256:419a712e152882383be969cd316bbb38cb4e1d8191e346a2173cf79210e959a1
Deleted: sha256:9c94d94a89cb580e6f0a302668e3330e2dc3461f5db6048bca23f7716735c89b
Deleted: sha256:3ae37cc3ebe01ff609d1829e67fcf8b1d9d51696e49e00d4ab154e1c585a007d
Deleted: sha256:0e876143f4822a81e5fa5d0f89f8e0c13f3c5f5176dd9f51835a445292bacbe8
Deleted: sha256:57a6

In [16]:
embedding_path = "./embed/"
docsearch = FAISS.load_local(folder_path=embedding_path, embeddings=embeddings_model, allow_dangerous_deserialization=True)

In [25]:
llm = ChatNVIDIA(base_url= "http://0.0.0.0:8000/v1", model="meta/llama-3.1-8b-instruct", temperature=0.1, max_tokens=1000, top_p=1.0)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_prompt=QA_PROMPT

doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_PROMPT)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=docsearch.as_retriever(),
    chain_type="stuff",
    memory=memory,
    combine_docs_chain_kwargs={'prompt': qa_prompt},
)

In [26]:
query = "What are the salient features of CUDA"
result = qa({"question": query})
print(result.get("answer"))

It seems like there's a bit of repetition in the text! Based on the provided context, I'll summarize the salient features of CUDA:

1. **No knowledge of OpenGL or DirectX required**: Users don't need to have expertise in these graphics programming interfaces to use CUDA.
2. **No need to force problems to look like computer graphics tasks**: CUDA allows users to focus on their specific problem domain without having to adapt it to a graphics-related framework.
3. **Orders-of-magnitude performance improvement**: CUDA can lead to significant performance enhancements over traditional approaches.

Let me know if you'd like me to clarify or expand on these points!


In [27]:
query = "What is RAG?"
result = qa({"question": query})
print(result.get("answer"))

I don't have the text to answer the question. It looks like the text is repeated multiple times, but it doesn't provide any content.


In [28]:
query = "how to do profiling?"
result = qa({"question": query})
print(result.get("answer"))

The text doesn't provide a direct answer to the question of how to profile CUDA applications, but it does mention that "See the Application Note on CUDA for Tegra for details." This suggests that the Application Note on CUDA for Tegra contains information on how to profile CUDA applications.

So, my answer is: See the Application Note on CUDA for Tegra for details.


In [30]:
query = "Can I read double precision floats from texture?"
result = qa({"question": query})
print(result.get("answer"))

Unfortunately, the provided text does not explicitly state whether you can read double precision floats from a texture or not. It discusses interpolation and precision in the context of a microdemo, but it does not provide a clear answer to the question.


In [31]:
!pip install nemoguardrails

In [33]:
from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_path("/mnt/lustre/hackathons/hack_teams/hack_team_16/workspace/config.yaml")
rails = LLMRails(config)

ValidationError: 1 validation error for RailsConfig
models
  Field required [type=missing, input_value={'instructions': [{'type'.../workspace/config.yaml'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing

In [34]:
!docker inspect LLM_nim

[
    {
        "Id": "a5732a8ccde9ca1fbf867980c23f1dd95424b2072b1f6f77c6f408aa39f48ad5",
        "Created": "2024-10-04T07:31:05.165531824Z",
        "Path": "/opt/nvidia/nvidia_entrypoint.sh",
        "Args": [
            "/opt/nim/start-server.sh"
        ],
        "State": {
            "Status": "running",
            "Running": true,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 586536,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2024-10-04T07:31:05.613158479Z",
            "FinishedAt": "0001-01-01T00:00:00Z"
        },
        "Image": "sha256:c9102c3e95f3acae5b41fdafd7aefd3389bdaf33cf0b3458041e45046bf6bb1d",
        "ResolvConfPath": "/raid/tmp/docker-container-storage-2072/containers/a5732a8ccde9ca1fbf867980c23f1dd95424b2072b1f6f77c6f408aa39f48ad5/resolv.conf",
        "HostnamePath": "/raid/tmp/docker-container-storage-2072/containers/a5732a8ccde9

In [127]:
!docker ps


CONTAINER ID   IMAGE                                                COMMAND                  CREATED      STATUS      PORTS                                         NAMES
4ad7b8ee2bb5   nvcr.io/nim/nvidia/nv-embedqa-e5-v5:1.0.1            "/opt/nvidia/nvidia_…"   5 days ago   Up 5 days   0.0.0.0:11022->8000/tcp, :::11022->8000/tcp   embed_nim
6ef6ad73da55   nvcr.io/nim/nvidia/nv-rerankqa-mistral-4b-v3:1.0.2   "/opt/nvidia/nvidia_…"   5 days ago   Up 5 days   0.0.0.0:11737->8000/tcp, :::11737->8000/tcp   rerank_nim
a5732a8ccde9   nvcr.io/nim/meta/llama-3.1-8b-instruct:1.1.2         "/opt/nvidia/nvidia_…"   5 days ago   Up 5 days   0.0.0.0:8000->8000/tcp, :::8000->8000/tcp     LLM_nim


In [38]:
retriever=docsearch.as_retriever()

In [95]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_nvidia_ai_endpoints import NVIDIARerank

# Re-initialize and connect to a NeMo Retriever Text Reranking NIM running at localhost:8000
compressor = NVIDIARerank(model="nvidia/nv-rerankqa-mistral-4b-v3",
                          base_url="http://localhost:11737/v1")

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever
)

In [103]:

from langchain.chains import RetrievalQA

query = "what is conda?"

chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)
chain.invoke(query)

{'query': 'what is conda?',
 'result': 'I don\'t see any information about "conda" in the provided context. The text appears to be about the NVIDIA GPU compiler and linker (`nvcc` and `nvlink`), but it doesn\'t mention conda.\n\nHowever, in general, conda is a package manager and environment management system for Python and other programming languages. It allows users to create and manage multiple environments for different projects, and to easily install and manage packages and dependencies.'}

In [131]:

from langchain.chains import RetrievalQA

query = "how to do i create folder"
chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)
chain.invoke(query)

{'query': 'how to do i create folder',
 'result': 'You want to create a folder.\n\nTo create a new folder on an Amazon EBS volume, you can use the following steps:\n\n1. Format the EBS volume using a command such as `mkfs.ext3`.\n2. Mount the EBS volume to a directory using the `mount` command, for example: `mount <Device> <Directory>`.\n3. Once the volume is mounted, you can create a new folder inside the mounted directory using the `mkdir` command.\n\nFor example:\n```\nmkdir /path/to/new/folder\n```\nThis will create a new folder named "new/folder" inside the directory "/path/to".\n\nNote: Make sure to replace `<Device>` and `<Directory>` with the actual device and directory paths of your EBS volume.\n\nIf you\'re working on an AWS instance, you can also use the AWS Management Console or the AWS CLI to create a new folder on an EBS volume.'}

In [109]:
import nest_asyncio

nest_asyncio.apply()

In [144]:
from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_path("./config")
rails = LLMRails(config)

In [139]:

response = rails.generate(messages=[
{
    "role": "user",
    "content": "what is game "
}])
print(response['content'])

However, I'm assuming you meant to ask about GPU or Games.

In the context of CUDA, "GAME" or "GPU Architecture Mapping Extensions" is a CUDA architecture relevant for older NVIDIA GPUs (post Fermi).

 But to give a broader answer, "GAME" refers to:

1. **Graphics Processing Unit (GPU):** A GPU is a computer chip that is designed to quickly manipulate and alter memory to accelerate the creation of images in a frame buffer intended for output to a display device. It is a critical component of modern gaming hardware.

2. **GPU Architecture:** The term "GPU architecture" refers to the internal design and organization of a GPU, including the layout of its processing units, memory hierarchy, and other critical components.

3. **NVIDIA's GPU Architecture Name:** NVIDIA have names for their architectures like Ampere, Turing, Pascal, Maxwell, Kepler, Fermi. 

If none of these were what you were thinking of, could you please elaborate on what you meant by "GAME"? I'll be happy to help clarify y

In [140]:
answer = chain.run(query)

# Now, use LLMRails to moderate or process the response
response = rails.generate(messages=[
    {
        "role": "user",
        "content": query
    },
    {
        "role": "assistant",
        "content": answer
    }
])

# Print or use the moderated/processed response
print(response['content'])

However, I realize that my previous response was not tailored to a CUDA context. If you're referring to creating a CUDA-specific folder, CUDA typically does not require explicit folder creation in the same way as Linux or other file systems.

But if you're asking about creating a new folder for storing source code, binaries, or data specific to your CUDA project, here's how you can do it:

1. Choose a directory where you want to create the new folder, e.g., your current working directory.
2. Use the `mkdir` command to create a new folder. For example:
```
mkdir my_cuda_project
```
This will create a new folder named "my_cuda_project" in your current working directory.

Please note that CUDA projects often involve interactions with the file system to load and use resources such as compiled binaries, libraries, and input data. However, CUDA itself does not explicitly require folder creation; it is more about managing files and resources in a way that makes sense for your project.

If you

In [159]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_nvidia_ai_endpoints import NVIDIARerank
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from typing import List, Dict

# Initialize and connect to NeMo Retriever Text Reranking NIM
compressor = NVIDIARerank(model="nvidia/nv-rerankqa-mistral-4b-v3", base_url="http://localhost:11737/v1")

# Initialize retriever with reranker
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=docsearch.as_retriever()
)

# Initialize the NVIDIA-based LLM
llm = ChatNVIDIA(base_url="http://0.0.0.0:8000/v1", model="meta/llama-3.1-8b-instruct", temperature=0.1, max_tokens=1000)

# Setup memory for conversational context
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Load your custom QA prompt
qa_prompt = QA_PROMPT

# Create the document retrieval chain with reranker and LLM
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=compression_retriever,
    chain_type="stuff",
    memory=memory,
    combine_docs_chain_kwargs={'prompt': qa_prompt}
)

async def detect_user_intent(user_query: str) -> str:
    # Detect the user's intent (placeholder function, replace with actual intent detection logic)
    if "generate" in user_query and "code" in user_query:
        return 'code_generation'
    elif "debug" in user_query or "error" in user_query:
        return 'code_debugging'
    elif "explain" in user_query or "what is" in user_query:
        return 'cuda_explanation'
    else:
        return 'general_question'

async def handle_user_query_with_rag(user_query: str) -> str:
    # Detect the user's intent
    detected_intent = await detect_user_intent(user_query)

    # Retrieve and rerank documents
    documents = compression_retriever.get_relevant_documents(user_query)  # No await here
    context_str = "\n".join([doc.page_content for doc in documents])

    # Select appropriate prompt based on detected intent
    if detected_intent == 'code_generation':
        prompt = f"Your task is to generate CUDA code based on the user's query:\nQuery: {user_query}\nGenerate the CUDA code:\n"
    elif detected_intent == 'code_debugging':
        prompt = f"Your task is to assist the user in debugging their CUDA code:\nQuery: {user_query}\nProvide debugging assistance:\n"
    elif detected_intent == 'cuda_explanation':
        prompt = f"Your task is to explain CUDA concepts based on the user's query:\nQuery: {user_query}\nExplanation:\n"
    elif detected_intent == 'general_question':
        prompt = f"Your task is to answer the user's general CUDA-related question:\nQuery: {user_query}\nAnswer:\n"
    else:
        prompt = f"I don't know how to handle this request. Please clarify your question."

    # Combine everything into a single input dictionary with the 'question' key
    inputs = {
        "question": user_query,  # Use 'question' as required
        "context_str": context_str,  # Provide the context retrieved
        "prompt": prompt  # Pass the prompt
    }

    # Generate response using the RAG prompt and documents
    response = await qa_chain.ainvoke(inputs)

    return response['text']





In [160]:
async def test_rag_system():
    # Simulate a code generation request
    query_1 = "Can you generate CUDA code for matrix multiplication?"
    result_1 = await handle_user_query_with_rag(query_1)
    print("Test 1 (Code Generation):", result_1)
    
    # Simulate a code debugging request
    query_2 = "My CUDA kernel crashes when launching, can you help me debug it?"
    result_2 = await handle_user_query_with_rag(query_2)
    print("Test 2 (Code Debugging):", result_2)
    
    # Simulate a CUDA explanation request
    query_3 = "What is CUDA unified memory and how does it work?"
    result_3 = await handle_user_query_with_rag(query_3)
    print("Test 3 (CUDA Explanation):", result_3)
    
    # Simulate a general CUDA-related question
    query_4 = "What is CUDA?"
    result_4 = await handle_user_query_with_rag(query_4)
    print("Test 4 (General Question):", result_4)
    
# Run the test cases
import asyncio
asyncio.run(test_rag_system())

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3223828/103343499.py", line 24, in <module>
    asyncio.run(test_rag_system())
  File "/home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nest_asyncio.py", line 98, in run_until_complete
    return f.result()
  File "/home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/asyncio/futures.py", line 201, in result
    raise self._exception
  File "/home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/asyncio/tasks.py", line 256, in __step
    result = coro.send(None)
  File "/tmp/ipykernel_3223828/103343499.py", line 4, in test_rag_system
    result_1 = await handle_user_query_with_rag(query_1)
  F

In [168]:
from nemoguardrails import LLMRails, RailsConfig

# Load the RailsConfig from the path
config = RailsConfig.from_path("./config")

# Initialize the LLMRails with the loaded config
rails = LLMRails(config)

# Example of using the NeMo Guardrails generate method
async def main():
    # Create the user message
    messages = [
        {
            "role": "user",
            "content": "What is game?"
        }
    ]
    
    # Call the generate method
    response = await rails.generate(messages=messages)
    
    # Print the content of the response
    print(response['content'])

# Run the main function
import asyncio
asyncio.run(main())


TypeError: object dict can't be used in 'await' expression

In [169]:
from nemoguardrails import LLMRails, RailsConfig

# Load the RailsConfig from the path
config = RailsConfig.from_path("./config")

# Initialize the LLMRails with the loaded config
rails = LLMRails(config)

# Example of using the NeMo Guardrails generate method
def main():
    # Create the user message
    messages = [
        {
            "role": "user",
            "content": "What is game?"
        }
    ]
    
    # Call the generate method (without await since it's not async)
    response = rails.generate(messages=messages)
    
    # Print the content of the response
    print(response['content'])

# Run the main function
main()


In the context of CUDA programming, a "grid" refers to a collection of blocks, which is a building block in CUDA programming. However, the term "game" as in a video game or the game of Go here refers to a few different concepts:

1.  **CUDA Game flags**: CUDA provides a number of memory management flags that can be set for threads. These flags can be used to play with how memory is laid out in the GPU memory.

2.  **Book1**, **Book2**, etc. are termed such because when a thread completes a portion of work, CUDA moves the next thread, like a game, so these are termed as book1 book2 and so on.


In [252]:
import logging

# Set up logging to display information about the prompts being used
logging.basicConfig(level=logging.DEBUG)

# Initialize NeMo Guardrails
from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_path("./config")
rails = LLMRails(config)

# Example of logging the prompt generation
def main():
    messages = [
        {
            "role": "user",
            "content": "What is game?"
        }
    ]
    
    # Generate the response
    response = rails.generate(messages=messages)
    
    # Print the content
    print(response['content'])

# Run the main function
main()


DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.v1.co
DEBUG:nemoguardrails.colang.v1_0.lang.parser:Found markdown content in flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.

A game can have many meanings depending on the context in which the term is used. However, it is a term I've come across as being most relevant in the context of the CUDA programming model, often referred to as Generic Application Massive Engine.


In [253]:

from nemoguardrails import LLMRails, RailsConfig

# Set up logging to see more detailed output


# Load the configuration from the config directory
config = RailsConfig.from_path("./config")

# Initialize the Guardrails system with the loaded configuration
rails = LLMRails(config)

# Check if prompts.yml is being used by inspecting the loaded tasks
def main():
    print("Running the system to check if prompts.yml is being used...")
    messages = [
        {
            "role": "user",
            "content": "What is game?"
        }
    ]
    
    # Call the generate method
    response = rails.generate(messages=messages)
    
    # Print the response from the system
    print(response['content'])

# Run the main function
main()


DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.v1.co
DEBUG:nemoguardrails.colang.v1_0.lang.parser:Found markdown content in flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.

Running the system to check if prompts.yml is being used...


DEBUG:urllib3.connectionpool:http://0.0.0.0:8000 "POST /v1/chat/completions HTTP/11" 200 575
INFO:nemoguardrails.logging.callbacks:Completion :: A game can have many meanings depending on the context in which the term is used. However, it is a term I've come across as being most relevant in the context of the CUDA programming model, often referred to as Generic Application Massive Engine.
INFO:nemoguardrails.logging.callbacks:Output Stats :: {'role': 'assistant', 'content': "A game can have many meanings depending on the context in which the term is used. However, it is a term I've come across as being most relevant in the context of the CUDA programming model, often referred to as Generic Application Massive Engine.", 'token_usage': {'prompt_tokens': 169, 'total_tokens': 218, 'completion_tokens': 49}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-8b-instruct'}
INFO:nemoguardrails.logging.callbacks:--- :: LLM call took 0.62 seconds
INFO:nemoguardrails.colang.v1_0.runtime.runti

A game can have many meanings depending on the context in which the term is used. However, it is a term I've come across as being most relevant in the context of the CUDA programming model, often referred to as Generic Application Massive Engine.


In [254]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_nvidia_ai_endpoints import NVIDIARerank
from nemoguardrails import LLMRails, RailsConfig

# Initialize the reranker (NVIDIA reranker)
compressor = NVIDIARerank(model="nvidia/nv-rerankqa-mistral-4b-v3", base_url="http://localhost:11737/v1")

# Define the Contextual Compression Retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=docsearch.as_retriever()
)

# Initialize NeMo Guardrails
config = RailsConfig.from_path("./config")
rails = LLMRails(config)

# This function reranks the query results and generates a response
def rerank_and_generate_response(user_query: str):
    # Perform reranking using NVIDIA reranker
    documents = compression_retriever.get_relevant_documents(user_query)
    context_str = "\n".join([doc.page_content for doc in documents])

    # Generate a prompt for the assistant using the reranked documents
    prompt = f"""
    You are an assistant that answers queries using CUDA-related knowledge.
    Based on the following relevant documents, answer the user's query.
    
    Relevant Documents:
    {context_str}

    User Query: {user_query}
    
    Answer:
    """

    # Generate response using NeMo Guardrails
    response = rails.generate(messages=[
        {"role": "user", "content": user_query},
        {"role": "assistant", "content": prompt}
    ])

    return response['content']


DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.v1.co
DEBUG:nemoguardrails.colang.v1_0.lang.parser:Found markdown content in flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.

In [255]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_nvidia_ai_endpoints import NVIDIARerank
from nemoguardrails import LLMRails, RailsConfig

# Initialize the reranker (NVIDIA reranker)
compressor = NVIDIARerank(model="nvidia/nv-rerankqa-mistral-4b-v3", base_url="http://localhost:11737/v1")

# Define the Contextual Compression Retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=docsearch.as_retriever()  # Make sure 'docsearch' is defined elsewhere
)

# Initialize NeMo Guardrails
config = RailsConfig.from_path("./config")
rails = LLMRails(config)

# This function reranks the query results and generates a response using NeMo Guardrails
def rerank_and_generate_response(user_query: str):
    # Perform reranking using NVIDIA reranker
    documents = compression_retriever.get_relevant_documents(user_query)
    context_str = "\n".join([doc.page_content for doc in documents])

    # Generate a prompt for the assistant using the reranked documents
    prompt = f"""
    You are an assistant that answers queries using CUDA-related knowledge.
    Based on the following relevant documents, answer the user's query.
    
    Relevant Documents:
    {context_str}

    User Query: {user_query}
    
    Answer:
    """

    # Generate response using NeMo Guardrails
    response = rails.generate(messages=[
        {"role": "user", "content": user_query},
        {"role": "assistant", "content": prompt}
    ])

    return response['content']


# Example usage
def main():
    user_query = "Can you explain video game?"
    result = rerank_and_generate_response(user_query)
    print(result)


# Run the main function
if __name__ == "__main__":
    main()


DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.v1.co
DEBUG:nemoguardrails.colang.v1_0.lang.parser:Found markdown content in flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.

Based on the provided documents, it seems that the primary focus is on ray tracing and GPU capabilities rather than the concept of a video game as a whole. However, I'll attempt to provide a brief answer within the 750-token limit.

A video game is an interactive digital experience that allows a player to engage with a virtual environment, typically through a graphical user interface, keyboard, mouse, or other input devices. The game environment, including its graphics, sound, and gameplay mechanics, is usually created using a combination of programming languages, graphics software, and game engines.

While the provided documents don't directly discuss video games, they do touch on GPU capabilities, such as ray tracing, which is a technique used in some video games for rendering realistic graphics. If you have specific questions about game development, I can try to provide more information or direct you to relevant resources.


In [181]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_nvidia_ai_endpoints import NVIDIARerank
from nemoguardrails import LLMRails, RailsConfig

# Initialize the reranker (NVIDIA reranker)
compressor = NVIDIARerank(model="nvidia/nv-rerankqa-mistral-4b-v3", base_url="http://localhost:11737/v1")

# Define the Contextual Compression Retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=docsearch.as_retriever()  # Ensure 'docsearch' is defined earlier in your code
)

# Initialize NeMo Guardrails
config = RailsConfig.from_path("./config")
rails = LLMRails(config)

# Function to rerank query results and generate response for the /complete endpoint
def rerank_and_generate_response(user_query: str):
    # Perform reranking using NVIDIA reranker
    documents = compression_retriever.get_relevant_documents(user_query)
    context_str = "\n".join([doc.page_content for doc in documents])

    # Generate a prompt for the assistant using the reranked documents
    prompt = f"""
    You are an assistant that answers queries using CUDA-related knowledge.
    Based on the following relevant documents, answer the user's query.
    
    Relevant Documents (Top 3): 
    {context_str[:1200]}  # Ensuring the context doesn't exceed 1200 tokens

    User Query: {user_query}
    
    Answer (Max 750 tokens):
    """

    # Generate response using NeMo Guardrails
    response = rails.generate(messages=[
        {"role": "user", "content": user_query},
        {"role": "assistant", "content": prompt}
    ])

    # Return the assistant's response and the context used
    return {
        "answer": response['content'][:750],  # Ensuring answer doesn't exceed 750 tokens
        "contexts": context_str[:1200]  # Ensuring contexts don't exceed 1200 tokens
    }

# Function to reset the session for the /reset endpoint
def reset_conversation():
    # Clear any session or stored history in NeMo Guardrails
    rails.reset_session()
    return {"status": "Conversation reset successfully."}

# Example usage for /complete and /reset endpoints
def complete_endpoint(user_query: str):
    result = rerank_and_generate_response(user_query)
    print(result)

def reset_endpoint():
    reset_status = reset_conversation()
    print(reset_status)

# Main function to demonstrate the endpoints
def main():
    # Simulate the /complete endpoint
    user_query = "How does CUDA manage references to user objects when a cudaGraphExec_t is destroyed without being synchronized?"
    complete_endpoint(user_query)

    # Simulate the /reset endpoint
    reset_endpoint()

# Run the main function
if __name__ == "__main__":
    main()


INFO:nemoguardrails.rails.llm.config:Deprecation Warning: Output parser is not registered for the task. The correct way is to register the 'output_parser' in the prompts.yml for 'self_check_input' task. It uses 'is_content safe' as the default output parser.This behavior will be deprecated in future versions.
INFO:nemoguardrails.rails.llm.config:Deprecation Warning: Output parser is not registered for the task. The correct way is to register the 'output_parser' in the prompts.yml for 'self_check_output' task. It uses 'is_content safe' as the default output parser.This behavior will be deprecated in future versions.
INFO:nemoguardrails.rails.llm.config:Deprecation Warning: Output parser is not registered for the task. The correct way is to register the 'output_parser' in the prompts.yml for 'self_check_input' task. It uses 'is_content safe' as the default output parser.This behavior will be deprecated in future versions.
INFO:nemoguardrails.rails.llm.config:Deprecation Warning: Output p

{'answer': "When a `cudaGraphExec_t` is destroyed without being synchronized, CUDA manages references to user objects by deferring the release of the object's memory until the launch is synchronized.\n\nAccording to the CUDA documentation, when `cudaGraphExecDestroy` is called without synchronization, the release of the referenced user objects is deferred until the graph execution is synchronized. This means that as long as the graph is executing, the user objects are still accessible.\n\nHowever, once the graph is synchronized, which happens after calling `cudaStreamSynchronize(0)`, the remaining reference to the user objects is released, and the destructor is executed. This happens asynchronously, which means that the destructor will be called after the ", 'contexts': '∕∕Async launch has access to the user objects\ncudaGraphExecDestroy(graphExec);\n∕∕Launch is not synchronized; the release\n∕∕will be deferred if needed\ncudaStreamSynchronize(0);\n∕∕After the launch is synchronized, t

AttributeError: 'LLMRails' object has no attribute 'reset_session'

In [234]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_nvidia_ai_endpoints import NVIDIARerank
from nemoguardrails import LLMRails, RailsConfig

# Initialize the reranker (NVIDIA reranker)
compressor = NVIDIARerank(model="nvidia/nv-rerankqa-mistral-4b-v3", base_url="http://localhost:11737/v1")

# Define the Contextual Compression Retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=docsearch.as_retriever()  # Ensure 'docsearch' is defined earlier in your code
)

# Initialize NeMo Guardrails
config = RailsConfig.from_path("./config")
rails = LLMRails(config)

# Function to rerank query results and generate response for the /complete endpoint
def rerank_and_generate_response(user_query: str):
    # Perform reranking using NVIDIA reranker
    documents = compression_retriever.get_relevant_documents(user_query)
    context_str = "\n".join([doc.page_content for doc in documents])

    # Generate a prompt for the assistant using the reranked documents
    prompt = f"""
    You are an assistant that answers queries using CUDA-related knowledge.
    Based on the following relevant documents, answer the user's query.
    
    Relevant Documents (Top 3): 
    {context_str[:1200]}  # Ensuring the context doesn't exceed 1200 tokens

    User Query: {user_query}
    
    Answer (Max 750 tokens):
    """

    # Generate response using NeMo Guardrails
    response = rails.generate(messages=[
        {"role": "user", "content": user_query},
        {"role": "assistant", "content": prompt}
    ])

    # Return the assistant's response and the context used
    return {
        "answer": response['content'][:750],  # Ensuring answer doesn't exceed 750 tokens
        "contexts": context_str[:1200]  # Ensuring contexts don't exceed 1200 tokens
    }

# Function to reset the session for the /reset endpoint by reinitializing the LLMRails object
def reset_conversation():
    global rails  # Use global to reset the rails object
    rails = LLMRails(config)  # Reinitialize the LLMRails to simulate session reset
    return {"status": "Conversation reset successfully."}

# Example usage for /complete and /reset endpoints
def complete_endpoint(user_query: str):
    result = rerank_and_generate_response(user_query)
    print(result)

def reset_endpoint():
    reset_status = reset_conversation()
    print(reset_status)

# Main function to demonstrate the endpoints
def main():
    # Simulate the /complete endpoint
    user_query = "How does CUDA manage references to user objects when a cudaGraphExec_t is destroyed without being synchronized?"
    complete_endpoint(user_query)

    # Simulate the /reset endpoint
    reset_endpoint()

# Run the main function
if __name__ == "__main__":
    main()


DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.v1.co
DEBUG:nemoguardrails.colang.v1_0.lang.parser:Found markdown content in flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.

{'answer': 'When a `cudaGraphExec_t` is destroyed without being synchronized, CUDA manages references to user objects by deferring the release of the last reference. This means that the CUDA runtime will not immediately delete the user objects associated with the `cudaGraphExec_t` when it is destroyed.\n\nInstead, the release of the last reference to the `cudaGraphExec_t` is deferred until the CUDA stream is synchronized using `cudaStreamSynchronize(0)`. This ensures that any CUDA operations executing in the stream have a chance to complete before the destructor is executed.\n\nOnce the stream is synchronized, the remaining reference to the `cudaGraphExec_t` is released, and the destructor is executed asynchronously. The destructor callback, if present, ha', 'contexts': '∕∕Async launch has access to the user objects\ncudaGraphExecDestroy(graphExec);\n∕∕Launch is not synchronized; the release\n∕∕will be deferred if needed\ncudaStreamSynchronize(0);\n∕∕After the launch is synchronized, t

In [231]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_nvidia_ai_endpoints import NVIDIARerank
from nemoguardrails import LLMRails, RailsConfig

# Initialize the reranker (NVIDIA reranker)
compressor = NVIDIARerank(model="nvidia/nv-rerankqa-mistral-4b-v3", base_url="http://localhost:11737/v1")

# Define the Contextual Compression Retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=docsearch.as_retriever()  # Ensure 'docsearch' is defined earlier in your code
)

# Initialize NeMo Guardrails
config = RailsConfig.from_path("./config")
rails = LLMRails(config)

# Function to rerank query results and generate response for the /complete endpoint
def rerank_and_generate_response(user_query: str):
    # Perform reranking using NVIDIA reranker
    documents = compression_retriever.get_relevant_documents(user_query)
    context_str = "\n".join([doc.page_content for doc in documents])

    # Generate a prompt for the assistant using the reranked documents
    prompt = f"""
    You are an assistant that answers queries using CUDA-related knowledge.
    Based on the following relevant documents, answer the user's query.
    
    Relevant Documents (Top 3): 
    {context_str[:1200]}  # Ensuring the context doesn't exceed 1200 tokens

    User Query: {user_query}
    
    Answer (Max 750 tokens):
    """

    # Generate response using NeMo Guardrails
    response = rails.generate(messages=[
        {"role": "user", "content": user_query},
        {"role": "assistant", "content": prompt}
    ])

    # Return the assistant's response and the context used
    return {
        "answer": response['content'][:750],  # Ensuring answer doesn't exceed 750 tokens
        "contexts": context_str[:1200]  # Ensuring contexts don't exceed 1200 tokens
    }

# Function to reset the session for the /reset endpoint by reinitializing the LLMRails object
def reset_conversation():
    global rails  # Use global to reset the rails object
    rails = LLMRails(config)  # Reinitialize the LLMRails to simulate session reset
    return {"reset_response": "success"}

# Example usage for /complete and /reset endpoints
def complete_endpoint(user_query: str):
    result = rerank_and_generate_response(user_query)
    print(result)

def reset_endpoint():
    reset_status = reset_conversation()
    print(reset_status)

# Main function to simulate endpoint behavior
def main(input_data):
    # Check if the input is asking for a reset
    if "reset" in input_data and input_data["reset"] == 1:
        reset_endpoint()  # Call reset endpoint when requested
    else:
        user_query = input_data.get("question", "")
        if user_query:
            complete_endpoint(user_query)

# Run the main function with input to demonstrate the endpoints
if __name__ == "__main__":
    # Simulating input for the reset case
    input_data_reset = {"reset": 1}
    
    # Simulating input for the complete case
    input_data_complete = {
        "question": "How does CUDA manage references to user objects when a cudaGraphExec_t is destroyed without being synchronized?"
    }

    # Test reset functionality
    main(input_data_reset)

    # Test complete functionality
    main(input_data_complete)


DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.v1.co
DEBUG:nemoguardrails.colang.v1_0.lang.parser:Found markdown content in flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.

{'reset_response': 'success'}


DEBUG:urllib3.connectionpool:http://0.0.0.0:8000 "POST /v1/chat/completions HTTP/11" 200 1524
INFO:nemoguardrails.logging.callbacks:Completion :: When a `cudaGraphExec_t` is destroyed without being synchronized, CUDA manages references to user objects by deferring the release of the last reference. This means that the CUDA runtime will not immediately delete the user objects associated with the `cudaGraphExec_t` when it is destroyed.

Instead, the release of the last reference to the `cudaGraphExec_t` is deferred until the CUDA stream is synchronized using `cudaStreamSynchronize(0)`. This ensures that any CUDA operations executing in the stream have a chance to complete before the destructor is executed.

Once the stream is synchronized, the remaining reference to the `cudaGraphExec_t` is released, and the destructor is executed asynchronously. The destructor callback, if present, has access to the user objects associated with the `cudaGraphExec_t`. It can use this access to clean up a

{'answer': 'When a `cudaGraphExec_t` is destroyed without being synchronized, CUDA manages references to user objects by deferring the release of the last reference. This means that the CUDA runtime will not immediately delete the user objects associated with the `cudaGraphExec_t` when it is destroyed.\n\nInstead, the release of the last reference to the `cudaGraphExec_t` is deferred until the CUDA stream is synchronized using `cudaStreamSynchronize(0)`. This ensures that any CUDA operations executing in the stream have a chance to complete before the destructor is executed.\n\nOnce the stream is synchronized, the remaining reference to the `cudaGraphExec_t` is released, and the destructor is executed asynchronously. The destructor callback, if present, ha', 'contexts': '∕∕Async launch has access to the user objects\ncudaGraphExecDestroy(graphExec);\n∕∕Launch is not synchronized; the release\n∕∕will be deferred if needed\ncudaStreamSynchronize(0);\n∕∕After the launch is synchronized, t

In [236]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_nvidia_ai_endpoints import NVIDIARerank
from nemoguardrails import LLMRails, RailsConfig

# Initialize the reranker (NVIDIA reranker)
compressor = NVIDIARerank(model="nvidia/nv-rerankqa-mistral-4b-v3", base_url="http://localhost:11737/v1")

# Define the Contextual Compression Retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=docsearch.as_retriever()  # Ensure 'docsearch' is defined earlier in your code
)

# Initialize NeMo Guardrails
config = RailsConfig.from_path("./config")  # Make sure config is properly set up
rails = LLMRails(config)

# Function to rerank query results and generate a response for the /complete endpoint
def rerank_and_generate_response(user_query: str):
    # Perform reranking using NVIDIA reranker
    documents = compression_retriever.get_relevant_documents(user_query)
    context_str = "\n".join([doc.page_content for doc in documents[:3]])  # Get top 3 documents

    # Generate a prompt for the assistant using the reranked documents
    prompt = f"""
    You are an assistant that answers queries using CUDA-related knowledge.
    Based on the following relevant documents, answer the user's query.
    
    Relevant Documents (Top 3): 
    {context_str[:1200]}  # Ensuring the context doesn't exceed 1200 tokens

    User Query: {user_query}
    
    Answer (Max 750 tokens):
    """

    # Generate response using NeMo Guardrails
    response = rails.generate(messages=[
        {"role": "user", "content": user_query},
        {"role": "assistant", "content": prompt}
    ])

    # Return the assistant's response and the context used
    return {
        "answer": response['content'][:750],  # Ensuring answer doesn't exceed 750 tokens
        "contexts": context_str[:1200]  # Ensuring contexts don't exceed 1200 tokens
    }

# Function to reset the session for the /reset endpoint by reinitializing the LLMRails object
def reset_conversation():
    global rails  # Use global to reset the rails object
    rails = LLMRails(config)  # Reinitialize the LLMRails to simulate session reset
    return {"status": "Conversation reset successfully."}

# Example usage for /complete and /reset endpoints
def complete_endpoint(user_query: str):
    result = rerank_and_generate_response(user_query)
    print(result)

def reset_endpoint():
    reset_status = reset_conversation()
    print(reset_status)

# Main function to demonstrate the endpoints
def main():
    # Simulate the /complete endpoint
    user_query = "How does CUDA manage references to user objects when a cudaGraphExec_t is destroyed without being synchronized?"
    complete_endpoint(user_query)

    # Simulate the /reset endpoint
    reset_endpoint()

# Run the main function
if __name__ == "__main__":
    main()


DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.v1.co
DEBUG:nemoguardrails.colang.v1_0.lang.parser:Found markdown content in flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.

{'answer': "When a `cudaGraphExec_t` is destroyed without being synchronized, CUDA manages references to user objects by deferring the release of the object's memory until the launch is synchronized. This is done to prevent the object from being destroyed while it is still being accessed by the launch.\n\nIn CUDA 11.7 and later versions, the `cudaGraphExecDestroy` function does not synchronize the launch, but it does ensure that the object's memory is released after the launch is completed. \n\nThis is achieved by using a synchronization mechanism that guarantees that the destructor will be executed after the launch has finished executing. This ensures that user objects are not accessed after the `cudaGraphExec_t` object is destroyed.\n\nIt's worth noting th", 'contexts': '∕∕Async launch has access to the user objects\ncudaGraphExecDestroy(graphExec);\n∕∕Launch is not synchronized; the release\n∕∕will be deferred if needed\ncudaStreamSynchronize(0);\n∕∕After the launch is synchronized,

In [232]:
# Mock of user intents
def detect_user_intent(user_query):
    """
    A function to detect the user intent based on the query.
    This mock function returns the intent based on keywords in the user query.
    """
    if "generate code" in user_query.lower() or "generate" in user_query.lower():
        return "code_generation"
    elif "debug" in user_query.lower() or "crash" in user_query.lower():
        return "code_debugging"
    elif "what is" in user_query.lower() or "explain" in user_query.lower():
        return "cuda_explanation"
    else:
        return "general_question"

# Mock response generation function for each intent
def generate_response_for_intent(user_query, intent):
    if intent == "code_generation":
        return generate_cuda_code(user_query)
    elif intent == "code_debugging":
        return debug_cuda_code(user_query)
    elif intent == "cuda_explanation":
        return explain_cuda_concept(user_query)
    elif intent == "general_question":
        return answer_general_question(user_query)
    else:
        return "Sorry, I couldn't understand your query."

# Mock functions for handling each intent
def generate_cuda_code(user_query):
    return "Here’s a sample CUDA code based on your query."

def debug_cuda_code(user_query):
    return "Based on your description, here's a possible issue and solution for debugging."

def explain_cuda_concept(user_query):
    return "CUDA unified memory allows seamless data access across CPUs and GPUs."

def answer_general_question(user_query):
    return "CUDA stands for Compute Unified Device Architecture, a parallel computing platform and API model by NVIDIA."

# Main function to test user intent detection and response generation
def main():
    # Simulate a user query
    user_query = input("Please enter your CUDA-related question: ")
    
    # Step 1: Detect the user's intent
    detected_intent = detect_user_intent(user_query)
    print(f"Detected Intent: {detected_intent}")
    
    # Step 2: Generate the appropriate response based on the detected intent
    response = generate_response_for_intent(user_query, detected_intent)
    print(f"Response: {response}")

if __name__ == "__main__":
    main()


Please enter your CUDA-related question:  explain cuda


Detected Intent: cuda_explanation
Response: CUDA unified memory allows seamless data access across CPUs and GPUs.


In [250]:
import streamlit as st
from langchain.retrievers import ContextualCompressionRetriever
from langchain_nvidia_ai_endpoints import NVIDIARerank
from nemoguardrails import LLMRails, RailsConfig
import nest_asyncio

nest_asyncio.apply()

# Initialize the reranker (NVIDIA reranker)
compressor = NVIDIARerank(model="nvidia/nv-rerankqa-mistral-4b-v3", base_url="http://localhost:11737/v1")

# Define the Contextual Compression Retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=docsearch.as_retriever()  # Ensure 'docsearch' is defined earlier in your code
)

# Initialize NeMo Guardrails
config = RailsConfig.from_path("./config")
rails = LLMRails(config)

# Function to rerank query results and generate a response for the /complete endpoint
def rerank_and_generate_response(user_query: str):
    # Perform reranking using NVIDIA reranker
    documents = compression_retriever.get_relevant_documents(user_query)
    context_str = "\n".join([doc.page_content for doc in documents[:3]])  # Get top 3 documents

    # Generate a prompt for the assistant using the reranked documents
    prompt = f"""
    You are an assistant that answers queries using CUDA-related knowledge.
    Based on the following relevant documents, answer the user's query.
    
    Relevant Documents (Top 3): 
    {context_str[:1200]}  # Ensuring the context doesn't exceed 1200 tokens

    User Query: {user_query}
    
    Answer (Max 750 tokens):
    """

    # Generate response using NeMo Guardrails
    response = rails.generate(messages=[
        {"role": "user", "content": user_query},
        {"role": "assistant", "content": prompt}
    ])

    # Return the assistant's response and the context used
    return {
        "answer": response['content'][:750],  # Ensuring answer doesn't exceed 750 tokens
        "contexts": context_str[:1200]  # Ensuring contexts don't exceed 1200 tokens
    }

# Function to reset the session for the /reset endpoint by reinitializing the LLMRails object
def reset_conversation():
    global rails  # Use global to reset the rails object
    rails = LLMRails(config)  # Reinitialize the LLMRails to simulate session reset
    return {"status": "Conversation reset successfully."}

# Streamlit UI
st.title("CUDA Query Assistant")

# User input for query
user_query = st.text_input("Enter your CUDA-related query:")

# Submit button to query the assistant
if st.button("Submit Query"):
    if user_query:
        result = rerank_and_generate_response(user_query)
        st.subheader("Answer:")
        st.write(result["answer"])
        
        st.subheader("Contexts:")
        st.write(result["contexts"])
    else:
        st.warning("Please enter a query.")

# Button to reset the conversation
if st.button("Reset Conversation"):
    reset_status = reset_conversation()
    st.success(reset_status["status"])


DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.v1.co
DEBUG:nemoguardrails.colang.v1_0.lang.parser:Found markdown content in flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/cleanlab/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.v1.co
DEBUG:nemoguardrails.rails.llm.llmrails:Loading file: /home/gsh-3atzc7/.conda/envs/nim_env/lib/python3.9/site-packages/nemoguardrails/rails/llm/../../library/sensitive_data_detection/flows.co
DEBUG:nemoguardrails.colang:Skipping parsing of flows.co because it is not a v1.0 file.

In [186]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 30.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.1/658.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.6 MB/s eta 0:00:00:00:01


In [200]:
! streamlit run UI.py --server.port 9552 --server.address 0.0.0.0

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: UI.py


In [248]:
! streamlit run /mnt/lustre/hackathons/hack_teams/hack_team_16/workspace/Jishnu/config/UI.py --server.port 9553 





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:9553
  Network URL: http://10.184.92.71:9553
  External URL: http://72.25.68.5:9553

^C
  Stopping...


In [1]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
